In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-22-srev-curri2/OML-order1-id4-2023-02-23_03-56-47.197869.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-22-srev-curri2/OML-order1-id4-2023-02-23_03-56-47.197904_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600,
    "pln": "1fc"
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-02-23 06:51:59,823 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2023-02-23 06:52:01,117 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2023-02-23 06:52:01,120 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
# label to task converter - hack, since normally we would just use the task token/identifier
def get_task_from_label(label_idx, task_dict):
    for task_idx, class_list in task_dict.items():
        if label_idx in class_list: return task_idx
    return -1
print(get_task_from_label(8, memory_buffer.task_dict))
# Find mode: https://stackoverflow.com/questions/10797819/finding-the-mode-of-a-list
def mode(array):
    return max(array, key = array.count)
# label list to task id
def get_task_from_label_list(label_list, task_dict):
    return mode([get_task_from_label(label, task_dict) for label in label_list])
print(get_task_from_label_list([1,2,32,1,4,2,0], memory_buffer.task_dict))

0
1


In [9]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [10]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [11]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.4712, accuracy = 0.8625, precision = 0.9143, recall = 0.8667, F1 score = 0.8808
Query set metrics: Loss = 1.0291, accuracy = 0.7500, precision = 0.8500, recall = 0.7867, F1 score = 0.7957
Query ID 1/475
Support set metrics: Loss = 0.5695, accuracy = 0.7625, precision = 0.8159, recall = 0.7767, F1 score = 0.7853
Query set metrics: Loss = 1.0971, accuracy = 0.6250, precision = 0.4800, recall = 0.6600, F1 score = 0.5291
Query ID 2/475
Support set metrics: Loss = 0.4859, accuracy = 0.8250, precision = 0.8482, recall = 0.8267, F1 score = 0.8267
Query set metrics: Loss = 0.9460, accuracy = 0.5625, precision = 0.5800, recall = 0.5833, F1 score = 0.5714
Query ID 3/475
Support set metrics: Loss = 0.4756, accuracy = 0.8375, precision = 0.8675, recall = 0.8400, F1 score = 0.8473
Query set metrics: Loss = 0.9696, accuracy = 0.5625, precision = 0.6633, recall = 0.5667, F1 score = 

Support set metrics: Loss = 0.3718, accuracy = 0.8500, precision = 0.8508, recall = 0.8467, F1 score = 0.8435
Query set metrics: Loss = 0.6092, accuracy = 0.7500, precision = 0.8250, recall = 0.7625, F1 score = 0.7667
Query ID 36/475
Support set metrics: Loss = 0.5063, accuracy = 0.8500, precision = 0.8968, recall = 0.8500, F1 score = 0.8634
Query set metrics: Loss = 0.8430, accuracy = 0.7500, precision = 0.6667, recall = 0.7167, F1 score = 0.6514
Query ID 37/475
Support set metrics: Loss = 0.4233, accuracy = 0.8500, precision = 0.8659, recall = 0.8467, F1 score = 0.8516
Query set metrics: Loss = 1.0124, accuracy = 0.5000, precision = 0.4533, recall = 0.4033, F1 score = 0.4167
Query ID 38/475
Support set metrics: Loss = 0.5102, accuracy = 0.8125, precision = 0.8638, recall = 0.8200, F1 score = 0.8354
Query set metrics: Loss = 0.8773, accuracy = 0.5000, precision = 0.6000, recall = 0.5567, F1 score = 0.5138
Query ID 39/475
Support set metrics: Loss = 0.4411, accuracy = 0.8375, precision

Query set metrics: Loss = 1.2559, accuracy = 0.5000, precision = 0.6667, recall = 0.5667, F1 score = 0.5313
Query ID 71/475
Support set metrics: Loss = 0.4689, accuracy = 0.8125, precision = 0.8479, recall = 0.8200, F1 score = 0.8279
Query set metrics: Loss = 1.2332, accuracy = 0.4375, precision = 0.3500, recall = 0.3667, F1 score = 0.3500
Query ID 72/475
Support set metrics: Loss = 0.6150, accuracy = 0.7250, precision = 0.7714, recall = 0.7300, F1 score = 0.7388
Query set metrics: Loss = 0.7678, accuracy = 0.6250, precision = 0.5700, recall = 0.5900, F1 score = 0.5758
Query ID 73/475
Support set metrics: Loss = 0.5427, accuracy = 0.7750, precision = 0.8179, recall = 0.7900, F1 score = 0.7925
Query set metrics: Loss = 0.7966, accuracy = 0.5625, precision = 0.5167, recall = 0.4967, F1 score = 0.5056
Query ID 74/475
Support set metrics: Loss = 0.5019, accuracy = 0.8250, precision = 0.8512, recall = 0.8333, F1 score = 0.8362
Query set metrics: Loss = 1.0544, accuracy = 0.6250, precision =

Query set metrics: Loss = 0.7003, accuracy = 0.8125, precision = 0.8333, recall = 0.8333, F1 score = 0.8262
Query ID 106/475
Support set metrics: Loss = 0.4933, accuracy = 0.8250, precision = 0.8563, recall = 0.8333, F1 score = 0.8346
Query set metrics: Loss = 1.3050, accuracy = 0.5000, precision = 0.5200, recall = 0.5667, F1 score = 0.5313
Query ID 107/475
Support set metrics: Loss = 0.5358, accuracy = 0.7750, precision = 0.7929, recall = 0.7867, F1 score = 0.7805
Query set metrics: Loss = 1.8912, accuracy = 0.3750, precision = 0.4333, recall = 0.4000, F1 score = 0.3905
Query ID 108/475
Support set metrics: Loss = 0.4794, accuracy = 0.8125, precision = 0.8462, recall = 0.8200, F1 score = 0.8228
Query set metrics: Loss = 1.2784, accuracy = 0.4375, precision = 0.4417, recall = 0.3967, F1 score = 0.3567
Query ID 109/475
Support set metrics: Loss = 0.4142, accuracy = 0.8250, precision = 0.8577, recall = 0.8267, F1 score = 0.8345
Query set metrics: Loss = 0.9987, accuracy = 0.5625, precisi

Query set metrics: Loss = 0.6945, accuracy = 0.8125, precision = 0.8433, recall = 0.8267, F1 score = 0.8114
Query ID 141/475
Support set metrics: Loss = 0.3674, accuracy = 0.8750, precision = 0.8836, recall = 0.8733, F1 score = 0.8761
Query set metrics: Loss = 1.1232, accuracy = 0.5625, precision = 0.5000, recall = 0.5500, F1 score = 0.5000
Query ID 142/475
Support set metrics: Loss = 0.5732, accuracy = 0.7750, precision = 0.8214, recall = 0.7867, F1 score = 0.7964
Query set metrics: Loss = 0.6709, accuracy = 0.8750, precision = 0.8000, recall = 0.8500, F1 score = 0.8048
Query ID 143/475
Support set metrics: Loss = 0.4302, accuracy = 0.8750, precision = 0.9025, recall = 0.8767, F1 score = 0.8848
Query set metrics: Loss = 0.7226, accuracy = 0.8125, precision = 0.8600, recall = 0.8600, F1 score = 0.8267
Query ID 144/475
Support set metrics: Loss = 0.4843, accuracy = 0.8000, precision = 0.8084, recall = 0.8000, F1 score = 0.8014
Query set metrics: Loss = 1.1316, accuracy = 0.5000, precisi

...

Support set metrics: Loss = 0.4760, accuracy = 0.8750, precision = 0.8995, recall = 0.8700, F1 score = 0.8669
Query set metrics: Loss = 0.3771, accuracy = 0.8750, precision = 0.9062, recall = 0.9062, F1 score = 0.8810
Query ID 313/475
Support set metrics: Loss = 0.5649, accuracy = 0.8500, precision = 0.8921, recall = 0.8700, F1 score = 0.8621
Query set metrics: Loss = 0.4210, accuracy = 0.8750, precision = 0.8958, recall = 0.9333, F1 score = 0.9028
Query ID 314/475
Support set metrics: Loss = 0.4571, accuracy = 0.8875, precision = 0.9089, recall = 0.8800, F1 score = 0.8862
Query set metrics: Loss = 0.4314, accuracy = 0.9375, precision = 0.8214, recall = 0.8571, F1 score = 0.8367
Query ID 315/475
Support set metrics: Loss = 0.5524, accuracy = 0.8500, precision = 0.8976, recall = 0.8500, F1 score = 0.8482
Query set metrics: Loss = 0.9403, accuracy = 0.8125, precision = 0.8571, recall = 0.7143, F1 score = 0.7619
Query ID 316/475
Support set metrics: Loss = 0.4743, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5166, accuracy = 0.8750, precision = 0.9000, recall = 0.8800, F1 score = 0.8653
Query set metrics: Loss = 0.5188, accuracy = 0.8125, precision = 0.8125, recall = 0.8125, F1 score = 0.7917
Query ID 348/475
Support set metrics: Loss = 0.5600, accuracy = 0.8750, precision = 0.9032, recall = 0.8600, F1 score = 0.8667
Query set metrics: Loss = 0.7691, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.5436, accuracy = 0.8750, precision = 0.8987, recall = 0.8800, F1 score = 0.8789
Query set metrics: Loss = 0.6767, accuracy = 0.7500, precision = 0.7857, recall = 0.7500, F1 score = 0.7238
Query ID 350/475
Support set metrics: Loss = 0.4773, accuracy = 0.9125, precision = 0.9409, recall = 0.9200, F1 score = 0.9164
Query set metrics: Loss = 0.4205, accuracy = 0.9375, precision = 1.0000, recall = 0.9583, F1 score = 0.9750
Query ID 351/475
Support set metrics: Loss = 0.4885, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5361, accuracy = 0.8250, precision = 0.9038, recall = 0.8300, F1 score = 0.8343
Query set metrics: Loss = 0.7233, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.5187, accuracy = 0.8625, precision = 0.9031, recall = 0.8600, F1 score = 0.8641
Query set metrics: Loss = 0.6583, accuracy = 0.8125, precision = 0.8571, recall = 0.8524, F1 score = 0.8127
Query ID 384/475
Support set metrics: Loss = 0.4847, accuracy = 0.9000, precision = 0.9040, recall = 0.8900, F1 score = 0.8872
Query set metrics: Loss = 1.2940, accuracy = 0.6875, precision = 0.6167, recall = 0.6833, F1 score = 0.6233
Query ID 385/475
Support set metrics: Loss = 0.5211, accuracy = 0.8625, precision = 0.8894, recall = 0.8700, F1 score = 0.8573
Query set metrics: Loss = 0.8889, accuracy = 0.7500, precision = 0.8125, recall = 0.7396, F1 score = 0.7488
Query ID 386/475
Support set metrics: Loss = 0.4557, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5067, accuracy = 0.8750, precision = 0.9016, recall = 0.8700, F1 score = 0.8759
Query set metrics: Loss = 0.5352, accuracy = 0.7500, precision = 0.8000, recall = 0.8000, F1 score = 0.7667
Query ID 418/475
Support set metrics: Loss = 0.4450, accuracy = 0.9125, precision = 0.9162, recall = 0.9000, F1 score = 0.9023
Query set metrics: Loss = 0.6845, accuracy = 0.6875, precision = 0.6667, recall = 0.6979, F1 score = 0.6280
Query ID 419/475
Support set metrics: Loss = 0.5019, accuracy = 0.9000, precision = 0.9393, recall = 0.9100, F1 score = 0.9100
Query set metrics: Loss = 0.8658, accuracy = 0.6250, precision = 0.7917, recall = 0.6667, F1 score = 0.6806
Query ID 420/475
Support set metrics: Loss = 0.5416, accuracy = 0.8125, precision = 0.8496, recall = 0.8100, F1 score = 0.8089
Query set metrics: Loss = 0.7790, accuracy = 0.8750, precision = 0.8148, recall = 0.8333, F1 score = 0.8074
Query ID 421/475
Support set metrics: Loss = 0.5344, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5065, accuracy = 0.8625, precision = 0.8975, recall = 0.8800, F1 score = 0.8785
Query set metrics: Loss = 0.2987, accuracy = 0.9375, precision = 0.9524, recall = 0.9524, F1 score = 0.9429
Query ID 453/475
Support set metrics: Loss = 0.5304, accuracy = 0.8750, precision = 0.9167, recall = 0.8800, F1 score = 0.8728
Query set metrics: Loss = 0.6064, accuracy = 0.8125, precision = 0.8958, recall = 0.8646, F1 score = 0.8571
Query ID 454/475
Support set metrics: Loss = 0.4586, accuracy = 0.8875, precision = 0.9262, recall = 0.8900, F1 score = 0.8974
Query set metrics: Loss = 0.5364, accuracy = 0.8750, precision = 0.8000, recall = 0.8500, F1 score = 0.8167
Query ID 455/475
Support set metrics: Loss = 0.4590, accuracy = 0.9125, precision = 0.9385, recall = 0.9100, F1 score = 0.9149
Query set metrics: Loss = 0.4155, accuracy = 0.8750, precision = 0.7333, recall = 0.7778, F1 score = 0.7500
Query ID 456/475
Support set metrics: Loss = 0.4922, accuracy = 0.8750, preci

In [12]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-inlr005-2023-02-22-srev-curri2/OML-order1-id4-2023-02-23_03-56-47.197869_update5_results.csv


In [13]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-inlr005-2023-02-22-srev-curri2/OML-order1-id4-2023-02-23_03-56-47.197869_update5_time_inference.csv
